# Assignment: Getting and Processing data

## Due: Tuesday the 22nd of November 2016 15:00 p.m.

Please name your Jupyter notebook using the following naming convention: ASSIGNMENT_3_FIRSTNAME_LASTNAME.ipynb 

Please send your assignment to `emiel.van.miltenburg@vu.nl`.

## Sentiment analysis using the NLTK

Sometimes authors in NLP implement their findings in the NLTK, so as to make their software available to everyone. This happened with the VADER-tool (Valence  Aware  Dictionary  for sEntiment Reasoning, [Hutto & Gilbert 2014](http://comp.social.gatech.edu/papers/icwsm14.vader.hutto.pdf), full code [here](http://www.nltk.org/_modules/nltk/sentiment/vader.html)) As the name suggests, you can use this tool to determine the sentiment of a text. Reading the paper, you can see the algorithm itself is really simple, but it manages to capture sentiment pretty well. (And yes, we know the acronym is horrible.)

In this assignment, we'll put this tool to the test. First you need to install it through the NLTK.

In [ ]:
# Make sure you're connected to the internet, and download the sentiment lexicon first.
import nltk
nltk.download('vader_lexicon')

In [ ]:
# Import the sentiment analyzer class.
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# And instantiate a sentiment analyzer object.
sid = SentimentIntensityAnalyzer()

So how do you use this tool? Basically, you can just call it with any sentence you like! Here's an example:

In [10]:
sentences = ["I am sad.",
             "I am quite happy."]

print('--------')
for sentence in sentences:
    scores = sid.polarity_scores(sentence)
    print(sentence)
    print(scores)
    print('--------')

--------
I am sad.
{'neu': 0.244, 'neg': 0.756, 'pos': 0.0, 'compound': -0.4767}
--------
I am quite happy.
{'neu': 0.334, 'neg': 0.0, 'pos': 0.666, 'compound': 0.6115}
--------


**Exercise 1**

Of course this is a really simple example. You can probably think of some more challenging phrases to enter. Before we'll use the sentiment analyzer to answer some research questions, let's first get some intuitions about what the tool does well, and where it makes mistakes. Try to come up with challenging examples for the tool to judge, and write down your findings.

* What does the sentiment analyzer do well?
* What are the weak points of this tool?

## Hackernews

The [Hackernews API](https://github.com/HackerNews/API) is a nice source of discussions between people, and of opiniated text. Those discussions have a tree-like structure, starting with some kind of news item (the *parent*), which gets a couple of responses (*children*), which in turn get responses of their own (more *children*):

                                 ,------ Response -------- Response
               ,------ Response /------- Response -------- Response
    News-item /------- Response -------- Response -------- Response -------- Response     ,------ Response
                                                  \------- Response ----------- Response <------- Response
                                                                                          \------ Response

We're going to write some functions to archive specific news items and their comments. At the same time, we'll be using a dictionary to cache all comments and items. Here's a list of the functions to write:

* `download_item(key, cache)` downloads an item and returns it as a dictionary.
* `download_tree(key, cache)` downloads a tree, starting from the item with identifier `key`.
* `download_user(username, cache)` downloads user data.

We'll start with [this item](https://news.ycombinator.com/item?id=12926684), a link to rollingstone.com about the death of Leonard Cohen. [Here](https://hacker-news.firebaseio.com/v0/item/12926684.json?print=pretty) is the JSON link from the API. It looks like this:

```python
{
  "by" : "joaomsa",
  "descendants" : 139,
  "id" : 12926684,
  "kids" : [ 12927149, 12926778, 12926940, 12928011, 12927831, 12926845, 12927000, 12928269, 12930113, 12926915, 12927700, 12926985, 12927085, 12927660, 12927349, 12926797, 12926751, 12927372, 12930833, 12928461, 12927964, 12926853, 12926826, 12926900, 12926917, 12927052, 12931153, 12926815, 12927962, 12928757, 12926958, 12926859, 12932982, 12928236, 12927116, 12927287, 12927062, 12930475, 12931778, 12927956, 12928730, 12926891, 12927006, 12926772, 12928207, 12931654, 12926935, 12927048, 12930683, 12927147, 12927375, 12928688, 12931045, 12931424, 12927715, 12929711, 12926941, 12927380, 12928737, 12927781, 12930724, 12927532, 12927786, 12928316, 12935676 ],
  "score" : 973,
  "time" : 1478828820,
  "title" : "Leonard Cohen Has Died",
  "type" : "story",
  "url" : "http://www.rollingstone.com/music/news/leonard-cohen-dead-at-82-w449792"
}
```

The first item in `kids`, [a comment](https://hacker-news.firebaseio.com/v0/item/12927149.json?print=pretty), looks like this:

```python
{
  "by" : "muhic",
  "id" : 12927149,
  "parent" : 12926684,
  "text" : "Sad for the artistic loss but also glad he died at peace after a rich life spent doing what he loved till the last moment. He joins a special list, alongside Hintjens who also passed recently, of those who manage to strip the dread from death and stress the importance of &#x27;tidying up&#x27; over passive acceptance as one enters the final days.<p><i>“The big change is the proximity to death,” he said. “I am a tidy kind of guy. I like to tie up the strings if I can. If I can’t, also, that’s O.K. But my natural thrust is to finish things that I’ve begun.”</i><p><i>“For some odd reason,” he went on, “I have all my marbles, so far. I have many resources, some cultivated on a personal level, but circumstantial, too: my daughter and her children live downstairs, and my son lives two blocks down the street. So I am extremely blessed. I have an assistant who is devoted and skillful. I have a friend like Bob and another friend or two who make my life very rich. So in a certain sense I’ve never had it better. . . . At a certain point, if you still have your marbles and are not faced with serious financial challenges, you have a chance to put your house in order. It’s a cliché, but it’s underestimated as an analgesic on all levels. Putting your house in order, if you can do it, is one of the most comforting activities, and the benefits of it are incalculable.”</i> [0]<p>[0] <a href=\"http:&#x2F;&#x2F;www.newyorker.com&#x2F;magazine&#x2F;2016&#x2F;10&#x2F;17&#x2F;leonard-cohen-makes-it-darker\" rel=\"nofollow\">http:&#x2F;&#x2F;www.newyorker.com&#x2F;magazine&#x2F;2016&#x2F;10&#x2F;17&#x2F;leonard-cohen-m...</a>",
  "time" : 1478832933,
  "type" : "comment"
}

```

And we can learn more about the author of this comment by [searching for their name](https://hacker-news.firebaseio.com/v0/user/muhic.json?print=pretty). Here's the result: a JSON file with all their other posts.

```python
{
  "created" : 1428067410,
  "id" : "muhic",
  "karma" : 226,
  "submitted" : [ 12927149, 12927138, 12782022, 12775146, 12775113, 12775075, 12752650, 12720395, 12715272, 12710137, 12641133, 12633663, 12576818, 12576795, 12576783, 12576758, 12576741, 12576730, 12576713, 12572213, 12572168, 12572115, 12563942, 12563846, 12475067, 12474951, 12474893, 12470497, 12426759, 12420573, 12378648, 12328278, 12242845, 12100179, 11928360, 11814853, 11713753, 11713721, 11187226, 11169690, 11064015, 10826141, 10826045, 10806055, 10750122, 10283593, 10272081, 10247645, 9315733 ]
}

```

Before we start, let's create an item cache and a user cache so that we never download data twice. We'll use dictionaries to represent the downloaded data.

In [11]:
item_cache = dict() # Keys will be item IDs, values will be dictionaries based on the JSON data.
user_cache = dict() # Keys will be usernames, values will be dictionaries based on the JSON data.

Now let's import the libraries we'll need to download the actual data:

In [ ]:
import json
import requests

The URL for an item looks like this: `https://hacker-news.firebaseio.com/v0/item/12926684.json`. From this, we can see that:

* There's a base url: `https://hacker-news.firebaseio.com/v0/item/`
* followed by an identifier: `12926684`
* and the extension: `.json`

**Exercise 2** Write a function, called `download_item(key)` that:

* takes an identifier (a string) as its argument
* downloads the corresponding item
* uses the `json` module to load the JSON data
* and then returns the loaded data.

Also remember to write a docstring to describe your function.

HINT: read through the Recipepuppy example in topic 4!

In [ ]:
# Test the function by running this cell:
leonard_cohen_article = download_item("12926684")

# Store the item in the cache. We don't want to download it again!
item_cache["12926684"] = leonard_cohen_article

# Print the title of the article.
print(leonard_cohen_article["title"])

**Caching**

To cache the results, we'll make use of the way variables refer to objects in Python. Here's a toy example.

In [13]:
toy_cache = dict()

def update_cache(cache):
    "Toy function that adds a test string to a dictionary."
    fake_item = {"title": "this is a fake item"}
    cache['1'] = fake_item
    print("We updated the dictionary!")

print(toy_cache)
update_cache(toy_cache)
print(toy_cache)

{}
We updated the dictionary!
{'1': {'title': 'this is a fake item'}}


Check out the step-by-step execution of the code [here](http://pythontutor.com/visualize.html#code=toy_cache%20%3D%20dict(%29%0A%0Adef%20update_cache(cache%29%3A%0A%20%20%20%20%22Toy%20function%20that%20adds%20a%20test%20string%20to%20a%20dictionary.%22%0A%20%20%20%20fake_item%20%3D%20%7B%22title%22%3A%20%22this%20is%20a%20fake%20item%22%7D%0A%20%20%20%20cache%5B'1'%5D%20%3D%20fake_item%0A%20%20%20%20print(%22We%20updated%20the%20dictionary!%22%29%0A%0Aprint(toy_cache%29%0Aupdate_cache(toy_cache%29%0Aprint(toy_cache%29&cumulative=false&curInstr=9&heapPrimitives=false&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false). What happens is that `toy_cache` initially is made to refer to an empty dictionary. Then, when `update_cache` is called, the variable `cache` is *made to refer to the same dictionary*. This means that `cache['1'] = fake_item` also has an effect on the dictionary that `toy_cache` is referring to.

**Exercise 3** Copy/paste the code you've written for `download_item(key)` below. Modify the function so that it has two arguments: `download_item(key, cache)`. The desired behavior of this function is as follows:

* When the function is called with a key that is already in the cache, return the associated item.
* When the function is called with a key that is not already in the cache, download the relevant item, add it to the cache, and return the item.

To test your function, add some `print` functions in both conditions to show that the function behaves differently for these two commands:

In [ ]:
# Redefine item_cache to make sure the test still works after running your function multiple times.
item_cache = dict()
item_cache["12926684"] = leonard_cohen_article

# Command 1: This item should already be in the cache:
download_item("12926684", item_cache)

# Command 2: This item shouldn't be in the cache yet.
download_item("12927149", item_cache)

**Exercise 4** Write a function called `download_user(username, cache)`. Its behavior should be similar to `download_item`, only for users. The relevant link is: `https://hacker-news.firebaseio.com/v0/user/USERNAME.json`.

**Downloading all the replies**

Now you've written the basic functions to download items and users, we can write a larger function to collect all the replies to the Leonard Cohen article. But at first glance, it's not very clear how to do that with this kind of structure:

                                 ,------ Response -------- Response
               ,------ Response /------- Response -------- Response
    News-item /------- Response -------- Response -------- Response -------- Response     ,------ Response
                                                  \------- Response ----------- Response <------- Response
                                                                                          \------ Response

How do you go through a structure like this? We'll use a to-do list: for the news item, we make a to-do list out of all its responses. These are the items we need to download and store in the cache. But those responses also have responses! So for each response, we extend our to-do list with all its responses. 

**Exercise 5** You don't have to write the entire function yourself. Just complete `download_tree` below.

In [ ]:
# Note that you don't have to start on the function!
# You can also first test pieces of your code here.

# E.g. how do you remove one of the elements from a list and make a variable refer to that element?
example_list = [1,2,3]
removed_element = example_list.some_method()
# See if it works
print(example_list, removed_element)

In [ ]:
def download_tree(key, cache):
    "Function that downloads all the responses for a given item, and returns a list of response IDs."
    # Define a list to collect all the keys.
    all_keys = []
    root = download_item(key, cache)
    
    # Here's the to do list. It starts out as a copy of the list of kids from the root.
    to_do = root['kids'][:]
    
    # This loop will keep running until there are no more things to download.
    while not len(to_do) == 0:
        # YOUR CODE HERE:
        # - Use a list method to remove one of the keys from the to do list,
        #   and assign that value to a variable called current_key.
        # - Add the current key to all_keys.
        # - Download the current item.
        # - Add its response keys to the to do list.
        # - Wait 2 seconds.
    return all_keys

**Putting it all together**

Now that you're able to download all the replies, you can start analyzing the data. It might be a good idea to save the data, so that you can load it in a future session.

In [ ]:
# Finish this code to store the item cache on your computer.
with open("item_cache.json",'w') as f:
    # Your code here

There's a lot we could do with this code and data:

* analyze online discussions
* create a dataset to see if you can determine who wrote which comment
* can you think of more possibilities?

**Intermezzo: sorting**

You can use the built-in function `sorted()` ([here](https://docs.python.org/3.6/library/functions.html#sorted)) or `list.sort()` to sort a list. We will use `sorted` as an example. This function takes an iterable and returns it as a sorted list. `list.sort` doesn't return anything, but rather sorts lists in-place. By default, the sorting algorithm determines the ordering based on the first element of whatever you want to sort (in this case tuples).

In [ ]:
some_list = [('a',4),('b',6),('c',1),('d',5)]
sorted_list = sorted(some_list)

# How is the list sorted?
print(sorted_list)

In [ ]:
# And we can sort in reverse order:
print(sorted_list, reversed=True)

Both `sorted` and `list.sort` have a keyword argument called `key`. This determines how the sorting function determines the order. 

The `key` argument can be any function, which means you can also supply the sorting algorithm with a function to select the *second* element of each object in the list. It's very common to use `lambda`-expressions for this. Here are some examples:

In [ ]:
sorted_list = sorted(some_list, 
                     key=lambda item:item[1]) # Use the element with index 1 to sort the list.

# How is the list sorted?
print(sorted_list)

In [ ]:
sorted_list = sorted(some_list, 
                     key=lambda item:item[0]) # Use the element with index 0 to sort the list.

# How is the list sorted?
print(sorted_list)

But you can achieve the same thing with the itemgetter() function. Many people find this clearer than using lambdas:

In [ ]:
from operator import itemgetter

sorted_list = sorted(some_list, 
                     key=itemgetter(0)) # Use the element with index 0 to sort the list.

# How is the list sorted?
print(sorted_list)

**Exercise 6** Use the VADER sentiment analysis tool from Exercise 1 to answer the following two questions:

1. What are the top-5 most positive comments in the Hackernews story about Leonard Cohen?
2. What are the top-5 most negative comments in the Hackernews story about Leonard Cohen?

Some hints:
* You can loop over the items in `item_cache` by using `for key, item in item_cache.items(): ...`
* You might need an additional list or dictionary to store the sentiment scores for each item.